In [1]:
import pandas as pd
from glob import glob
import itertools
from collections import Counter
import numpy as np
import numpy
import math
import matplotlib.pylab as plt
import os
import networkx as nx
from pypinyin import lazy_pinyin
import sys
from collections import defaultdict, Counter
import networkx as nx
import numpy as np
import pylab as plt
import math
#import matplotlib.cm as cm
import statsmodels.api as sm
from os import listdir
from scipy.stats.stats import pearsonr
import json
import random
import itertools
from copy import deepcopy
import time
import scipy as sp
from scipy.sparse import csr_matrix
import matplotlib.cm as cm
from numpy.random import choice
import itertools
from sklearn import manifold
import operator
import itertools
from datetime import datetime as dt
import matplotlib.mlab as mlab
from scipy.stats import norm
import re
from scipy import stats
from scipy.spatial import Voronoi
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from pypinyin import lazy_pinyin

/Users/xuhuimin/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [ ]:
# choose all the users
columns = 'user_id，sex，age，occupation_name，education_name，user_opentime，brand_name，call_duration_m，gprs_flow，\
call_fee，gprs_fee，databusiness_fee，order_name，brand_chn，model_chn，screensize，operation_sys，terminal_price，\
dept_county_name，dept_name'.split('，')

basename=pd.read_csv('./basename.csv', encoding='gbk', names=columns)

users=basename['user_id'].unique()

column_list = 'user_id,access_mode_id,logic_area_name,lac,ci,longtitude,latitude,busi_name,busi_type_name,\
app_name,app_type_name,start_time,up_pack,down_pack,up_flow,down_flow,site_name,site_channel,cont_app_id,\
cont_classify_id,cont_type_id,acce_url'.split(',')

for user in users:
    path='./allusers/'+'user_'+str(user)+'.csv'
    #print(path)
    if os.path.isfile(path)==True:
        
        userdata=pd.read_csv(path,names=column_list)
        userdata=userdata[['user_id','longtitude','latitude','app_name','app_type_name','start_time']]

        with open('./siteexample1/allusers/' + str(user) +'.csv', 'a+') as f:
            userdata.to_csv(f,header=False)

In [9]:
# choose the users with high house price & location records>3
basename_clean=pd.read_csv('./basename_clean.csv')
basename_clean=basename_clean[basename_clean['house_price'].astype('str')!='nan']
basename_clean=basename_clean[basename_clean['location']>3]
users=basename_clean['user_id'].unique()
# print(len(users))
column_list = 'user_id,access_mode_id,logic_area_name,lac,ci,longtitude,latitude,busi_name,busi_type_name,\
app_name,app_type_name,start_time,up_pack,down_pack,up_flow,down_flow,site_name,site_channel,cont_app_id,\
cont_classify_id,cont_type_id,acce_url'.split(',')

for user in users:
    path='./allusers/'+'user_'+str(user)+'.csv'
    #print(path)
    if os.path.isfile(path)==True:
        
        userdata=pd.read_csv(path,names=column_list)
        userdata=userdata[['user_id','longtitude','latitude','app_name','app_type_name','start_time']]

        with open('./siteexample1/allusers/' + str(user) +'.csv', 'a+') as f:
            userdata.to_csv(f,header=False)

44742


/Users/xuhuimin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# choose the users with terminal price & fees

In [2]:
basename_clean=pd.read_csv('./basename_clean.csv')
basename_clean=basename_clean[(basename_clean['terminal_price'].astype('str')!='nan')&(basename_clean['fees'].astype('str')!='nan')]
users=basename_clean['user_id'].unique()
# print(len(users))

for user in users:
    path='./siteexample1/allusers_complete/'+str(user)+'.csv'
    #print(path)
    if os.path.isfile(path)==True:
        
        userdata=pd.read_csv(path,names=['user_id','longtitude','latitude','app_name','app_type_name','start_time'])

        with open('./siteexample1/allusers/' + str(user) +'.csv', 'a+') as f:
            userdata.to_csv(f,header=False)

49315


/Users/xuhuimin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# attention

In [ ]:
# first put all the unique nodes together, second all pair combinations together,third pair combinations for every user
files=glob('./siteexample1/allusers/*')
for f in files:
    user=f.split('/')[-1].split('.')[0]
    userdata=pd.read_csv(f,names=['user_id','longtitude','latitude','app_name','app_type_name','start_time'])
    
    userdata=userdata[userdata['app_name']!='其他']
    userdata=userdata[['user_id','app_name']]
    userdata=userdata.drop_duplicates()
    
#     userdata.to_csv('./siteexample1/node.csv',header=False,mode='a+')
    app_list=userdata['app_name'].values
    pair=list(itertools.combinations(app_list,2))
    data=pd.DataFrame(pair,columns=['node1','node2'])

    data['node3']=data['node1'].map(lambda x: ''.join(lazy_pinyin(str(x))))
    data['node4']=data['node2'].map(lambda x: ''.join(lazy_pinyin(str(x))))
    for i in range(len(data.values)):
        if data.values[i][2]>data.values[i][3]:
            data.values[i][0],data.values[i][1]=data.values[i][1],data.values[i][0]
    
    data=data[['node1','node2']]
    data.to_csv('./siteexample1/pair.csv',header=False,mode='a+')
    
    with open('./siteexample1/pair/' + str(user) +'.csv', 'a+') as f:
        data.to_csv(f,header=False)

In [ ]:
# observed values
app_obs=pd.read_csv('./siteexample1/pair.csv',names=['node1','node2'],index_col=0)
app_obs['pair']=app_obs['node1'].astype('str')+' '+app_obs['node2'].astype('str')
counterdict=Counter(app_obs['pair'].values)
counterdict=pd.DataFrame(list(counterdict.items()),columns=['pair','app_obs'])
counterdict.to_csv('./siteexample1/observed_value.csv')    

In [ ]:
app_df = pd.read_csv('./siteexample1/observed_value.csv', index_col = 0)

app_df['node1'] = app_df['pair'].apply(lambda x: x.split(' ')[0])
app_df['node2'] = app_df['pair'].apply(lambda x: x.split(' ')[1])

app_df = app_df[['node1','node2','app_obs']]


From=[]
To=[]
for ii in app_df.index:
    i,j,w= app_df.node1[ii],app_df.node2[ii], app_df.app_obs[ii]
    x,y=zip(*[[i,j]]*w)
    From+=x
    To+=y

D=defaultdict(lambda:0)
for x,y in zip(From,To):
    D[(x,y)]+=1
D=dict(D)

# expected value
DD=defaultdict(lambda:[])
for i in range(10):
    D1=defaultdict(lambda:0)
    To1=np.random.permutation(To)
    for x,y in zip(From,To1):
        D1[(x,y)]+=1
    for x,y in D1:
        DD[(x,y)].append(D1[(x,y)])
DD=dict(DD)

# z score
d=[]
for x,y in D:
    if (x,y) in DD:
        m=np.mean(DD[(x,y)]) 
        sd=np.std(DD[(x,y)])
        if sd==0:
            sd=1
        w=D[(x,y)]
        z1=w-m
        z=(w-m+0.0)/sd
        d.append([x,y,w,z])
data = pd.DataFrame(d,columns=['node1','node2','weight','z_score'])
data['pair']=data['node1'].astype('str')+' '+data['node2'].astype('str')
data.to_csv('./siteexample1/observed_value.csv',header=False,index=False)

In [ ]:
# put z_score into every user-file
z_score=pd.read_csv('./siteexample1/observed_value.csv',names=['node1','node2','weight','z_score','pair'])
z_score=z_score[['pair','z_score']]
diction=dict(z_score.values)

files=glob('./siteexample1/pair/*')

for f in files:
    data=pd.read_csv(f,names=['node1','node2'])
    data['pair']=data['node1'].astype('str')+' '+data['node2'].astype('str')
    data['z_score']=data['pair'].apply(lambda x: diction[x]
                                  if x in diction
                                   else np.nan)
    data.to_csv(f)

In [ ]:
# four categories group by z_score
files=glob('./siteexample1/pair/*')
lista=[]
listb=[]
listc=[]
for f in files:
    data=pd.read_csv(f)
    if len(data['z_score'])>0:
        user_id=f.split('/')[-1].split('.')[0]
        lista.append(user_id)
        list1=list(data[data['z_score'].notnull()]['z_score'].values)
        a=numpy.array(list1)
        median=numpy.median(a)
        listb.append(median)
        pertentile_10=numpy.percentile(a,10)
        listc.append(pertentile_10)
df=pd.DataFrame([lista,listb,listc]).T
df.columns=['user_id','median','pertentile_10']
df.to_csv('./siteexample1/zscore_category.csv')

In [ ]:
# add category

# df=pd.read_csv('./siteexample1/zscore_category.csv',index_col=0)
# df=df[~df.isin([numpy.nan, numpy.inf, -numpy.inf]).any(1)]

# high_low=df[(df['median']>df['median'].median())&(df['pertentile_10']>0)].index
# df.ix[high_low,'category']='high_low'


# low_low=df[(df['median']<=df['median'].median())&(df['pertentile_10']>0)].index
# df.ix[low_low,'category']='low_low'

# low_high=df[(df['median']<=df['median'].median())&(df['pertentile_10']<=0)].index
# df.ix[low_high,'category']='low_high'

# high_high=df[(df['median']>df['median'].median())&(df['pertentile_10']<=0)].index
# df.ix[high_high,'category']='high_high'

# df=df[['user_id','median','pertentile_10','category']]
# df.to_csv('./siteexample1/zscore_category.csv')

# mobility

In [ ]:
# first put all the unique nodes together, second all pair combinations together,third pair combinations for every user
files=glob('./siteexample1/allusers/*')
for f in files:
    user=f.split('/')[-1].split('.')[0]
    userdata=pd.read_csv(f,names=['user_id','longtitude','latitude','app_name','app_type_name','start_time'])
    userdata[userdata['longtitude'].astype('str')!='nan']
    userdata=userdata[userdata['latitude'].astype('str')!='nan']
    userdata['location']=(userdata['longtitude'].astype('str')+','+userdata['latitude'].astype('str')).values
    userdata=userdata[userdata['app_name']!='其他']
    userdata=userdata[['user_id','location']]
    userdata=userdata.drop_duplicates()
    
#     userdata.to_csv('./siteexample1/location/node.csv',header=False,mode='a+')
    
    app_list=userdata['location'].values
    pair=list(itertools.combinations(app_list,2))
    data=pd.DataFrame(pair)
    
    for i in range(len(data.values)):
        if data.values[i][0].split(',')[0]+data.values[0][0].split(',')[1]>data.values[i][1].split(',')[0]+data.values[0][0].split(',')[1]:
            data.values[i][0],data.values[i][1]=data.values[i][1],data.values[i][0]
    
    data.to_csv('./siteexample1/location/pair.csv',header=False,mode='a+')
    
    with open('./siteexample1/location/pair/' + str(user) +'.csv', 'a+') as f:
        data.to_csv(f,header=False)

In [ ]:
# observed values
app_obs=pd.read_csv('./siteexample1/location/pair.csv',names=['node1','node2'],index_col=0)
app_obs['pair']=app_obs['node1'].astype('str')+' '+app_obs['node2'].astype('str')
counterdict=Counter(app_obs['pair'].values)
counterdict=pd.DataFrame(list(counterdict.items()),columns=['pair','app_obs'])
counterdict.to_csv('./siteexample1/location/observed_value.csv')  

In [ ]:
app_df = pd.read_csv('./siteexample1/location/observed_value.csv', index_col = 0)

app_df['node1'] = app_df['pair'].apply(lambda x: x.split(' ')[0])
app_df['node2'] = app_df['pair'].apply(lambda x: x.split(' ')[1])

app_df = app_df[['node1','node2','app_obs']]


From=[]
To=[]
for ii in app_df.index:
    i,j,w= app_df.node1[ii],app_df.node2[ii], app_df.app_obs[ii]
    x,y=zip(*[[i,j]]*w)
    From+=x
    To+=y

D=defaultdict(lambda:0)
for x,y in zip(From,To):
    D[(x,y)]+=1
D=dict(D)

# expected value
DD=defaultdict(lambda:[])
for i in range(10):
    D1=defaultdict(lambda:0)
    To1=np.random.permutation(To)
    for x,y in zip(From,To1):
        D1[(x,y)]+=1
    for x,y in D1:
        DD[(x,y)].append(D1[(x,y)])
DD=dict(DD)

# z score
d=[]
for x,y in D:
    if (x,y) in DD:
        m=np.mean(DD[(x,y)]) 
        sd=np.std(DD[(x,y)])
        if sd==0:
            sd=1
        w=D[(x,y)]
        z1=w-m
        z=(w-m+0.0)/sd
        d.append([x,y,w,z])
data = pd.DataFrame(d,columns=['node1','node2','weight','z_score'])
data['pair']=data['node1'].astype('str')+' '+data['node2'].astype('str')
data.to_csv('./siteexample1/location/observed_value.csv',header=False,index=False)

In [ ]:
# put z_score into every user-file
z_score=pd.read_csv('./siteexample1/location/observed_value.csv',names=['node1','node2','weight','z_score','pair'])
z_score=z_score[['pair','z_score']]
diction=dict(z_score.values)

files=glob('./siteexample1/location/pair/*')

for f in files:
    data=pd.read_csv(f,names=['node1','node2'])
    data['pair']=data['node1'].astype('str')+' '+data['node2'].astype('str')
    data['z_score']=data['pair'].apply(lambda x: diction[x]
                                  if x in diction
                                   else np.nan)
    data.to_csv(f)

In [ ]:
# four categories group by z_score
files=glob('./siteexample1/location/pair/*')
lista=[]
listb=[]
listc=[]
for f in files:
    data=pd.read_csv(f)
    if len(data[data['z_score'].notnull()])>0:
        user_id=f.split('/')[-1].split('.')[0]
        lista.append(user_id)
        list1=list(data[data['z_score'].notnull()]['z_score'].values)
        a=numpy.array(list1)
        median=numpy.median(a)
        listb.append(median)
        pertentile_10=numpy.percentile(a,10)
        listc.append(pertentile_10)
df=pd.DataFrame([lista,listb,listc]).T
df.columns=['user_id','median','pertentile_10']
df.to_csv('./siteexample1/location/zscore_category.csv')

In [ ]:
# add category

# df=pd.read_csv('./siteexample1/location/zscore_category.csv',index_col=0)
# df=df[~df.isin([numpy.nan, numpy.inf, -numpy.inf]).any(1)]

# high_low=df[(df['median']>df['median'].median())&(df['pertentile_10']>0)].index
# df.ix[high_low,'category']='high_low'


# low_low=df[(df['median']<=df['median'].median())&(df['pertentile_10']>0)].index
# df.ix[low_low,'category']='low_low'

# low_high=df[(df['median']<=df['median'].median())&(df['pertentile_10']<=0)].index
# df.ix[low_high,'category']='low_high'

# high_high=df[(df['median']>df['median'].median())&(df['pertentile_10']<=0)].index
# df.ix[high_high,'category']='high_high'

# df=df[['user_id','median','pertentile_10','category']]
# df.to_csv('./siteexample1/location/zscore_category.csv')